<a href="https://colab.research.google.com/github/vaishjah3/ML-Algorithms/blob/main/Residual_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [12]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs=25
batch_size=100
learning_rate=0.01

In [13]:
transform=transforms.Compose([transforms.Pad(4),
                              transforms.RandomHorizontalFlip(),
                              transforms.RandomCrop(32),
                              transforms.ToTensor()])

In [14]:
train_dataset=torchvision.datasets.CIFAR10(root='./data',
                                           download=True,
                                           train=True,
                                           transform=transform)
test_dataset=torchvision.datasets.CIFAR10(root='./data',
                                          download=True,
                                          train=False,
                                          transform=transform)

train_loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                         batch_size=batch_size,
                                         shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=batch_size,
                                        shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
def conv3x3(in_channels, out_channels, stride=1):
  return nn.Conv2d(in_channels, out_channels, kernel_size=3,stride=stride, padding=1, bias=False)

class ResidualBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride=1, downsample=None):
    super(ResidualBlock, self).__init__()
    self.conv1=conv3x3(in_channels, out_channels, stride)
    self.bn1=nn.BatchNorm2d(out_channels)
    self.relu=nn.ReLU(inplace=True)
    self.conv2=conv3x3(out_channels, out_channels)
    self.bn2=nn.BatchNorm2d(out_channels)
    self.downsample=downsample

  def forward(self,x):
    residual=x
    out=self.conv1(x)
    out=self.bn1(out)
    out=self.relu(out)
    out=self.conv2(out)
    out=self.bn2(out)
    if self.downsample:
      residual=self.downsample(x)
    out+=residual
    out=self.relu(out)
    return out


In [16]:
class ResNet(nn.Module):
  def __init__(self, block, layers, num_classes=10):
    super(ResNet, self).__init__()
    self.in_channels=16
    self.conv=conv3x3(3, 16)
    self.bn=nn.BatchNorm2d(16)
    self.relu=nn.ReLU(inplace=True)
    self.layer1=self.make_layer(block, 16, layers[0])
    self.layer2=self.make_layer(block, 32, layers[1],2)
    self.layer3=self.make_layer(block, 64, layers[2],2)
    self.avg_pool=nn.AvgPool2d(8)
    self.fc=nn.Linear(64, num_classes)

  def make_layer(self, block, out_channels, blocks, stride=1):
    downsample=None
    if (stride!=1) or (self.in_channels !=out_channels):
      downsample=nn.Sequential(conv3x3(self.in_channels, out_channels, stride),
                               nn.BatchNorm2d(out_channels))
    layers=[]
    layers.append(block(self.in_channels, out_channels, stride,downsample))
    self.in_channels=out_channels
    for i in range(1, blocks):
      layers.append(block(out_channels, out_channels))
    return nn.Sequential(*layers)

  def forward(self,x):
    out=self.conv(x)
    out=self.bn(out)
    out=self.relu(out)
    out=self.layer1(out)
    out=self.layer2(out)
    out=self.layer3(out)
    out=self.avg_pool(out)
    out=out.view(out.size(0), -1)
    out=self.fc(out)
    return out


In [17]:
model=ResNet(ResidualBlock, [2,2,2]).to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
decay=0
model.train()
for epoch in range(num_epochs):
  if (epoch+1)%20==0:
    decay+=1
    optimizer.param_groups[0]['lr']=learning_rate *(0.09**decay)
    print("The new learning rate is {}".format(optimizer.param_groups[0]['lr']))


  for i, (images, labels) in enumerate(train_loader):
    images=images.to(device)
    labels=labels.to(device)
    output=model(images)
    loss=criterion(output ,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) %100==0:
      print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
            .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

Epoch [1/25], Step [100/500] Loss: 0.2843
Epoch [1/25], Step [200/500] Loss: 0.1918
Epoch [1/25], Step [300/500] Loss: 0.1588
Epoch [1/25], Step [400/500] Loss: 0.1016
Epoch [1/25], Step [500/500] Loss: 0.2090
Epoch [2/25], Step [100/500] Loss: 0.1256
Epoch [2/25], Step [200/500] Loss: 0.1005
Epoch [2/25], Step [300/500] Loss: 0.1227
Epoch [2/25], Step [400/500] Loss: 0.1775
Epoch [2/25], Step [500/500] Loss: 0.2394
Epoch [3/25], Step [100/500] Loss: 0.1671
Epoch [3/25], Step [200/500] Loss: 0.1849
Epoch [3/25], Step [300/500] Loss: 0.1519
Epoch [3/25], Step [400/500] Loss: 0.1979
Epoch [3/25], Step [500/500] Loss: 0.0893
Epoch [4/25], Step [100/500] Loss: 0.1773
Epoch [4/25], Step [200/500] Loss: 0.2477
Epoch [4/25], Step [300/500] Loss: 0.1662
Epoch [4/25], Step [400/500] Loss: 0.0954
Epoch [4/25], Step [500/500] Loss: 0.3105
Epoch [5/25], Step [100/500] Loss: 0.3486
Epoch [5/25], Step [300/500] Loss: 0.2068
Epoch [5/25], Step [400/500] Loss: 0.1370
Epoch [5/25], Step [500/500] Loss:

In [23]:
model.eval()
with torch.no_grad():
  correct=0
  total=0
  for imgaes, labels in test_loader:
    images=images.to(device)
    labels=labels.to(device)
    outputs=model(images)
    _,predicted=torch.max(output.data, 1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()

  print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 10.25 %
